# Heart Disease Detection

    Referred Paper:
    https://pmc.ncbi.nlm.nih.gov/articles/PMC9687844/#_ad93_

    Referred MRI Data Set:
    https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data

Author's Model Replication

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import display

## Importing Heart Diseases Data and reshaping dataset for model [Filled NA values with Median].

In [ ]:
# Load dataset (UCI Heart Disease Dataset transformed)
file_path = "data.csv"
columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "target"]
# df = pd.read_csv(url, names=columns, na_values='?')
df = pd.read_csv(file_path, names=columns, na_values='?')

# Replace nan values with mean
df_mean = df.median()
df.fillna(df_mean, inplace=True)
# Handle missing values
df.dropna(inplace=True)

# Convert categorical features
label_encoders = {}
for col in ['ca', 'thal']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Convert target variable (binary classification: 0 (no disease) vs. 1 (disease))
df['target'] = (df['target'] > 0).astype(int)

# Split dataset
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Print data shape
print("Training Data Shape:", X_train.shape)
print("Testing Data Shape:", X_test.shape)

Creating Author's 1D Deep CNN model

In [ ]:
# Model V1 (As per the paper DCNN architecture)
def build_model_v1():

    dropout_rate = 0.03  # 3% dropout

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(13, 1)),  # Adjusted to 13 features
        tf.keras.layers.Conv1D(32, kernel_size=3, activation='elu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(64, kernel_size=3, activation='elu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),

        # Fully connected layers (4 layers with 128 neurons)
        tf.keras.layers.Dense(128, activation='elu'),
        tf.keras.layers.Dropout(dropout_rate),  # 3% dropout
        tf.keras.layers.Dense(128, activation='elu'),
        tf.keras.layers.Dropout(dropout_rate),  # 3% dropout
        tf.keras.layers.Dense(128, activation='elu'),
        tf.keras.layers.Dropout(dropout_rate),  # 3% dropout
        tf.keras.layers.Dense(128, activation='elu'),
        tf.keras.layers.Dropout(dropout_rate),  # 3% dropout

        # Fully connected layers (3 layers with 64 neurons)
        tf.keras.layers.Dense(64, activation='elu'),
        tf.keras.layers.Dropout(dropout_rate),  # 3% dropout
        tf.keras.layers.Dense(64, activation='elu'),
        tf.keras.layers.Dropout(dropout_rate),  # 3% dropout
        tf.keras.layers.Dense(64, activation='elu'),
        tf.keras.layers.Dropout(dropout_rate),  # 3% dropout

        # Output layer
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile model with Nadam optimizer
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
                loss='binary_crossentropy', metrics=['accuracy'])

    print("Model Summary:\n")
    model.summary()
    print("\n")
    return model

Creating our own 1D CNN BMK Model (less complex)

In [ ]:
# Model V2 (RELU activation model)
def build_model_v2():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(13, 1)),  # Input layer with 13 features (1D)
        tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),  # 1D Convolutional layer with 32 filters
        tf.keras.layers.MaxPooling1D(pool_size=2),  # Max pooling layer to reduce dimensionality

        tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),  # Another convolutional layer with 64 filters
        tf.keras.layers.MaxPooling1D(pool_size=2),  # Another max pooling layer

        tf.keras.layers.Flatten(),  # Flatten the 2D output into 1D
        tf.keras.layers.Dense(128, activation='relu'),  # Fully connected layer with 128 neurons
        tf.keras.layers.Dropout(0.05),  # Dropout layer to prevent overfitting
        tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
    ])
    # Compile the model with Nadam optimizer, binary crossentropy loss, and accuracy metric
    # model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
    #               loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='MeanSquaredError', metrics=['accuracy'])

    print("Model Summary:\n")
    model.summary()
    print("\n")
    return model

Running Author's model with transformed data set where we replaced nan values with median.

In [ ]:
# Reshape input data for Conv1D
X_train = X_train.reshape(-1, 13, 1)
X_test = X_test.reshape(-1, 13, 1)

# Train model
model = build_model_v1()

# to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=16, verbose=1)
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

class_weight = {0: 1, 1: 10}  # Adjust based on your dataset
history =  model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weight)
# history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
# print(classification_report(y_test, y_pred))

These functions are used to plot accuracies, loss, history and ROC Curves. These is also a function that handles the comparison logic between V1 and V2.

In [ ]:
# Plot training & validation loss
def plot_training_loss(history):
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training vs. Validation Loss')
    plt.tight_layout()
    # plt.savefig('plots/loss_test.png')
    plt.show()
plot_training_loss(history)

# Plot training & validation accuracy
def plot_training_history(history):
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training vs. Validation Accuracy')
    plt.tight_layout()
    # plt.savefig('plots/accuracy_test.png')
    plt.show()

# Confusion Matrix
def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Disease', 'Disease'], yticklabels=['No Disease', 'Disease'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    # plt.savefig('plots/confusion_matrix_test.png')
    plt.show()

# Plot ROC Curve
def plot_roc_curve(y_test, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc*100:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.tight_layout()
    # plt.savefig('plots/roc_curve_test.png')
    plt.show()

# Function to plot the comparison of both models
def plot_comparisons(history_v1, history_v2, y_test_v1, y_pred_v1, y_test_v2, y_pred_v2, y_pred_prob_v1, y_pred_prob_v2, save=True):
    # Plot Loss Comparison
    plt.figure(figsize=(10, 6))
    plt.plot(history_v1.history['loss'], label='V1 Train Loss', color='blue')
    plt.plot(history_v1.history['val_loss'], label='V1 Val Loss', color='cyan', linestyle='--')
    plt.plot(history_v2.history['loss'], label='V2 Train Loss', color='red')
    plt.plot(history_v2.history['val_loss'], label='V2 Val Loss', color='orange', linestyle='--')
    plt.title('Training and Validation Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    if save:
        plt.savefig('plots/loss_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Plot Accuracy Comparison
    plt.figure(figsize=(10, 6))
    plt.plot(history_v1.history['accuracy'], label='V1 Train Accuracy', color='blue')
    plt.plot(history_v1.history['val_accuracy'], label='V1 Val Accuracy', color='cyan', linestyle='--')
    plt.plot(history_v2.history['accuracy'], label='V2 Train Accuracy', color='red')
    plt.plot(history_v2.history['val_accuracy'], label='V2 Val Accuracy', color='orange', linestyle='--')
    plt.title('Training and Validation Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    if save:
        plt.savefig('plots/accuracy_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Plot Confusion Matrix Comparison
    cm_v1 = confusion_matrix(y_test_v1, y_pred_v1)
    cm_v2 = confusion_matrix(y_test_v2, y_pred_v2)
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))
    sns.heatmap(cm_v1, annot=True, fmt='d', cmap='Blues', xticklabels=['No Disease', 'Disease'], yticklabels=['No Disease', 'Disease'], ax=axs[0])
    axs[0].set_title('Model V1 Confusion Matrix')
    axs[0].set_xlabel('Predicted')
    axs[0].set_ylabel('Actual')
    sns.heatmap(cm_v2, annot=True, fmt='d', cmap='Blues', xticklabels=['No Disease', 'Disease'], yticklabels=['No Disease', 'Disease'], ax=axs[1])
    axs[1].set_title('Model V2 Confusion Matrix')
    axs[1].set_xlabel('Predicted')
    axs[1].set_ylabel('Actual')
    plt.tight_layout()
    if save:
        plt.savefig('plots/confusion_matrix_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Plot ROC Curve Comparison
    fpr_v1, tpr_v1, _ = roc_curve(y_test_v1, y_pred_prob_v1)
    fpr_v2, tpr_v2, _ = roc_curve(y_test_v2, y_pred_prob_v2)
    roc_auc_v1 = auc(fpr_v1, tpr_v1)
    roc_auc_v2 = auc(fpr_v2, tpr_v2)

    plt.figure(figsize=(10, 6))
    plt.plot(fpr_v1, tpr_v1, color='blue', lw=2, label=f'V1 ROC curve (area = {roc_auc_v1*100:.2f}%)')
    plt.plot(fpr_v2, tpr_v2, color='red', lw=2, label=f'V2 ROC curve (area = {roc_auc_v2*100:.2f}%)')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.title('ROC Curve Comparison')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.tight_layout()
    if save:
        plt.savefig('plots/roc_curve_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

The performance metrics function creates a report for out V1 Model

In [ ]:
def performance_metrics(y_test, y_pred):
    # Generate classification report for the current model
    report = classification_report(y_test, y_pred, output_dict=True)

    # Extract overall metrics
    metrics = {
        'Accuracy': report['accuracy'],
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1 Score': report['weighted avg']['f1-score']
    }

    # Create a DataFrame for metrics
    metrics_df = pd.DataFrame(metrics, index=['Model V1']).T

    # Export the metrics table to a CSV file
    # metrics_df.to_csv('model_v1_metrics.csv', index=True)

    # Print the metrics DataFrame
    print("Model V1 Metrics:\n")
    print(metrics_df)

Finally plotting the properties and performance metrics

In [ ]:
plot_training_history(history)
plot_confusion_matrix(y_test, y_pred)
plot_roc_curve(y_test, y_pred_prob)

performance_metrics(y_test, y_pred)

# Construction and Implementation of BMK Model (V2) and Comparing it with Author's (V1) Model

## BMK Model - Bhanuse, Majd, and Khanna model

Here we don't fill out the NaN values, we just remove all the empty or bad data sets.

In [ ]:
# Re-Load the dataset (UCI Heart Disease Dataset)
file_path = "data/data.csv"
columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "target"]
df = pd.read_csv(file_path, names=columns, na_values='?')  # Replace '?' with NaN

# Handle missing values by dropping rows with any NaN values
df.dropna(inplace=True)

# Convert categorical features ('ca' and 'thal') to numeric using Label Encoding
label_encoders = {}
for col in ['ca', 'thal']:
    le = LabelEncoder()  # Initialize label encoder
    df[col] = le.fit_transform(df[col])  # Convert categorical labels to numeric
    label_encoders[col] = le  # Store the encoder for future use

# Convert target variable to binary classification (0 = no disease, 1 = disease)
df['target'] = (df['target'] > 0).astype(int)  # Convert to 0 or 1

# Split the dataset into features (X) and target (y)
X = df.drop('target', axis=1)  # Features (drop the 'target' column)
y = df['target']  # Target variable (disease/no disease)

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit and transform the training data
X_test = scaler.transform(X_test)  # Only transform the test data (use the training set's scaler)

# Reshape input data for Conv1D: The model expects data of shape (samples, time steps, features)
X_train = X_train.reshape(-1, 13, 1)  # Reshape training data to 3D
X_test = X_test.reshape(-1, 13, 1)  # Reshape test data to 3D

In [ ]:
# Print the shapes of the training and testing data
print("Training Data Shape:", X_train.shape)
print("Testing Data Shape:", X_test.shape)

This function has the logic for getting a model as a function parameter and then it trains and runs the model.

In [ ]:
# Function to train and evaluate a model
def train_and_evaluate_model(model_func):
    # Initialize model
    model = model_func()

    # Train the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1, class_weight={0: 1, 1: 10})

    # Predict probabilities and threshold for binary predictions
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)

    return history, y_pred, y_pred_prob

This function handles the logic to compare the performance metrics of V1 (Author's) and V2 (Our Own) Models.

In [ ]:
def compare_performance_metrics(y_test, y_pred_v1, y_pred_v2):
    # Generate classification reports for both models
    report_v1 = classification_report(y_test, y_pred_v1, output_dict=True)
    report_v2 = classification_report(y_test, y_pred_v2, output_dict=True)

    # Extract overall metrics for both models
    metrics_v1 = {
        'Accuracy': report_v1['accuracy'],
        'Precision': report_v1['weighted avg']['precision'],
        'Recall': report_v1['weighted avg']['recall'],
        'F1 Score': report_v1['weighted avg']['f1-score']
    }
    metrics_v2 = {
        'Accuracy': report_v2['accuracy'],
        'Precision': report_v2['weighted avg']['precision'],
        'Recall': report_v2['weighted avg']['recall'],
        'F1 Score': report_v2['weighted avg']['f1-score']
    }

    # Create a DataFrame for comparison
    comparison_df = pd.DataFrame([metrics_v1, metrics_v2], index=['Model V1', 'Model V2']).T

    # Export the comparison table to a CSV file
    comparison_df.to_csv('model_comparison.csv', index=True)

    # Print the comparison DataFrame
    # print("Comparison Table:\n")
    display(comparison_df)

Finally, we train both the models. Plot the prediction results and performance metrics. Then finally compare them.

In [ ]:
# Train and evaluate both models
history_v1, y_pred_v1, y_pred_prob_v1 = train_and_evaluate_model(build_model_v1)
history_v2, y_pred_v2, y_pred_prob_v2 = train_and_evaluate_model(build_model_v2)

# Plot comparisons for both models
plot_comparisons(history_v1, history_v2, y_test, y_pred_v1, y_test, y_pred_v2, y_pred_prob_v1, y_pred_prob_v2, save=False)

# Compare performance metrics for both models
compare_performance_metrics(y_test, y_pred_v1, y_pred_v2)

Brain_MRI_Analysis using 2D and 1D CNN:

Original file is located at
    https://colab.research.google.com/drive/1dDWU8_tQ7pDdh3JnmmhJ3JtvdiIoszUP

    Referred Paper:
    https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-023-02114-6

    Referred MRI Data Set:
    https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri

First, we import all the dependencies that we need to perform the 2D CNN analysis on the Brain MRI Images.

In [4]:
import numpy as np
import random
import os

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

This is to unzip the images folder that we get from the Kaggle wesbite that we have mentioned above.

In [ ]:
!unzip archive.zip

This function rescales the images to 80*80 pixels, converts the image to grayscale and then add it to the image directory.

In [6]:
def process_image(jpg_image, image_directory, image_labels, type, label):
    jpg_image = jpg_image.convert("L")
    jpg_resized_image = jpg_image.resize((80, 80))

    # Convert to NumPy array
    image_array = np.array(jpg_resized_image, dtype=np.float32)  # FITS prefers float32
    # Normalization
    image_array = image_array / 255

    # rotated_image = np.rot90(image_array, axes=(1, 0))
    # flipped_image = np.flipud(image_array)

    image_directory[type].append(image_array)
    image_labels.append(label)

This function imports all the images and then calls the process function for transformation and storing thema as NumPy arrays.

In [14]:
# Load the JPG images
def import_images(file_directories, class_directory, training_images_directory, training_labels):
    # Loop through all files in the Training folder
    for file in file_directories:
        file_directory = "Training/" + file

        for filename in os.listdir(file_directory):
            file_path = os.path.join(file_directory, filename)

            type = None
            if 'glioma' in file_path:
                type = 'glioma'
            elif 'meningioma' in file_path:
                type = 'meningioma'
            elif 'pituitary' in file_path:
                type = 'pituitary'
            else:
                type = 'healthy'

            label = class_directory[type]

            # Check if the file is a JPG image
            if filename.lower().endswith('.jpg'):
                # Open the image correctly with the full path
                jpg_image = Image.open(file_path)  # Convert to grayscale

                # Process image to rescale and then add it onto the training directory
                process_image(jpg_image, training_images_directory, training_labels, type, label)

    for file in file_directories:
        file_directory = "Testing/" + file

        for filename in os.listdir(file_directory):
            file_path = os.path.join(file_directory, filename)

            type = None
            if 'glioma' in file_path:
                type = 'glioma'
            elif 'meningioma' in file_path:
                type = 'meningioma'
            elif 'pituitary' in file_path:
                type = 'pituitary'
            else:
                type = 'healthy'

            label = class_directory[type]

            # Check if the file is a JPG image
            if filename.lower().endswith('.jpg'):
                jpg_image = Image.open(file_path)  # Convert to grayscale

                # Process image to rescale and then add it onto the training directory
                process_image(jpg_image, training_images_directory, training_labels, type, label)

    return training_images_directory, training_labels

This is the 2D CNN model that we created to process, train and then predict the tumors or no tumors from the MRI images that we have with us.

In [8]:
# Building the 2D-CNN Model to train BRAIN MRI Images
def build_model_2D_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(80, 80, 1)),
        tf.keras.layers.Conv2D(64, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.Conv2D(64, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),  # Max pooling layer to reduce dimensionality
        tf.keras.layers.Dropout(0.1),  # Dropout layer to prevent overfitting

        tf.keras.layers.Conv2D(32, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(32, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),  # Max pooling layer to reduce dimensionality
        tf.keras.layers.Dropout(0.1),  # Dropout layer to prevent overfitting

        tf.keras.layers.Conv2D(16, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(16, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),  # Max pooling layer to reduce dimensionality
        tf.keras.layers.Dropout(0.1),  # Dropout layer to prevent overfitting

        tf.keras.layers.Conv2D(8, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.Conv2D(8, kernel_size=(2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),  # Max pooling layer to reduce dimensionality
        tf.keras.layers.Dropout(0.1),  # Dropout layer to prevent overfitting

        # The dense layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024),                # No activation here!
        tf.keras.layers.BatchNormalization(),      # Batch Normalization before activation
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.2),              # Dropout after activation
        tf.keras.layers.Dense(4, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print("Model V2 Summary:\n")
    model.summary()
    print("\n")
    return model

For our curiosity, we created a 1D CNN model similar to V2 model for heart disease to see how effectively that model can train compared with 2D CNN. Amazingly, we got very similar results.

In [9]:
# Building the 1D CNN Model to train BRAIN MRI Images
def build_model_1D_cnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((6400, 1), input_shape=(80, 80, 1)),  # Input layer reshaped to handle 1D 6400 pixel values of an image from 80 * 80 in 2D
        tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),  # Convolutional layer with 128 filters
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=2),  # Initial max pooling layer
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),  # Another convolutional layer with 64 filters
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=2),  # Another max pooling layer
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),   # Another convolutional layer with 64 filters
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=2),  # Max pooling layer to reduce dimensionality
        tf.keras.layers.Dropout(0.1),

        # The dense layers
        tf.keras.layers.Flatten(),  # Flatten the 2D output into 1D
        tf.keras.layers.Dense(128, activation='relu'),  # Fully connected layer with 128 neurons
        tf.keras.layers.Dropout(0.2),  # Dropout layer to prevent overfitting
        tf.keras.layers.Dense(4, activation='softmax')  # Output layer with softmax activation for multi-class classification
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print("Model V2 Summary:\n")
    model.summary()
    print("\n")
    return model

This is the code that initialize the model and then finally performs training and testing on the images, and get predictions.

In [10]:
def train_and_evaluate_model(model_func, training_data, training_label, testing_data, testing_label, class_weight):
    # Initialize model
    model = model_func()

    # Train the model
    history = model.fit(training_data, training_label, epochs=20, batch_size=16, validation_data=(testing_data, testing_label), verbose=1, class_weight = class_weights)

    # Predict probabilities and threshold for multi-class predictions
    y_pred_prob = model.predict(testing_data)

    return history, y_pred_prob

This snippet has all the plotting functions to visualize our accuracies, losses and ROC curves for both training and validations.

In [11]:
# Confusion matrix for multi-classification: Visualize true positives, false positives, true negatives, and false negatives
def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)  # Compute confusion matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Tumor', 'glioma', 'meninglioma', 'pituatary'], yticklabels=['No Tumor', 'glioma', 'meninglioma', 'pituatary'])  # Plot confusion matrix as heatmap
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    # plt.savefig("plots/confusion_matrix.png", dpi=300)
    plt.show()

From here, we have started initializing variables necessary to run the code.

In [12]:
# These are the 4 file directories or tumors that we study
file_directories = ["glioma_tumor", "meningioma_tumor", "pituitary_tumor", "no_tumor"]  # Directory of the file

# Classes of tumors and healthy brain mapped to numeric labels
class_directory = {
    'glioma': 1,
    'meningioma': 2,
    'pituitary': 3,
    'healthy': 0
}

# Store each image and it's correct label
labels = []

directory = {
    'glioma': [],
    'meningioma': [],
    'pituitary': [],
    'healthy': []
}

In this snippet, we import all the images, split the images into training and testing data sets and then sets class weights based on the number of each cases we have in our data sets.

In [15]:
# Importing MRI Images from the directory
training_images_directory, training_labels = import_images(file_directories, class_directory, directory, labels)

# Generate Training Data:
training_data = []
for key in training_images_directory:
    training_data = training_data + training_images_directory[key]

# Converting Training Data from list np.array
training_data = np.array(training_data, dtype='float32')
training_labels = np.array(training_labels, dtype='float32')

# Splitting properties
split_ratio = 0.1
random_int = random.randint(1, 100)

# Split the data set into training and testing
training_data, testing_data, training_labels, testing_labels = train_test_split(training_data, training_labels, test_size=split_ratio, random_state=random_int)

# Based on the size of training data set, compute weights to set priorities
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1, 2, 3]),
    y=training_labels
)

# Convert class weights into dictionary (0 - given the most as it's healthy brain case)
class_weights = dict(enumerate(class_weights))

Before training the model, we check if everything is correct or something isn't done correctly.

In [16]:
# Check the data sets and class weights
print(training_data.shape)
print(training_labels.shape)
print(testing_data.shape)
print(testing_labels.shape)
print(class_weights)

(2937, 80, 80)
(2937,)
(327, 80, 80)
(327,)
{0: np.float64(1.6389508928571428), 1: np.float64(0.8782894736842105), 2: np.float64(0.8772401433691757), 3: np.float64(0.8998161764705882)}


Finally, we run the model that we have built and get all the metrics.

In [ ]:
# 2D CNN: Finally, calling the function to train and then validate our model
history_mri, testing_pred_prob_mri = train_and_evaluate_model(build_model_2D_cnn, training_data, training_labels, testing_data, testing_labels, class_weights)

# 1D CNN (V2): Finally, calling the function to train and then validate our model
# history_mri, testing_pred_prob_mri = train_and_evaluate_model(build_model_1D_cnn, training_data, training_labels, testing_data, testing_labels, class_weights)

# print(testing_pred_prob_mri)

# Generating the prediction array (getting the most high probability value label from each case)
predictions = []
for probs in testing_pred_prob_mri:
  max_ind = 0
  val = probs[0]
  for i in range(1, len(probs)):
    if probs[i] > val:
      max_ind = i
      val = probs[i]

  predictions.append(max_ind)

predictions = np.array(predictions)

Time for some plotting.

In [ ]:
# Plotting the metrics received from running the models.
plot_confusion_matrix(testing_labels, predictions)
plot_training_loss(history_mri)
plot_training_history(history_mri)